In [403]:
import hmac
import hashlib
import requests
import time
import urllib.parse
import datetime
import pyupbit
import pandas as pd
import datetime
import numpy as np
import pickle

In [404]:
API_KEY = 'WNfy5TCeABGgzB2r5aplShWYPZ3YXUwQR35DmQHe4fE'
SECRET_KEY = 'OAa2yT19b9ZaD8E2WycMfpcW8V4Wn5mse6zW_z3vdCI'
base_url = "https://api.korbit.co.kr"

##   서명생성

In [405]:
def create_hmac_sha256_signature(SECRET_KEY, query_string):
    key_bytes = SECRET_KEY.encode()
    message_bytes = query_string.encode()
    hmac_hash = hmac.new(key_bytes, message_bytes, hashlib.sha256)
    return hmac_hash.hexdigest()

##    계좌 정보 가져오기

In [406]:
def get_krw_balance():
    timestamp = int(time.time() * 1000)

    params = {
        "timestamp": str(timestamp)
    }
    query_string = urllib.parse.urlencode(params)

    signature = create_hmac_sha256_signature(SECRET_KEY, query_string)
    params["signature"] = signature
    query_string = urllib.parse.urlencode(params)

    url = f"{base_url}/v2/balance?{query_string}"

    headers = {
        "X-KAPI-KEY": API_KEY
    }

    try:
        response = requests.get(url, headers=headers)
        response_data = response.json()
        
        if response_data and response_data.get('success'):
            balances = response_data['data']
            for asset in balances:
                if asset['currency'] == 'krw':
                    krw_balance = asset['balance']
                    krw_balance_integer = int(float(krw_balance))  # 정수로 변환
                    print("KRW Balance:", krw_balance_integer)
                    return krw_balance_integer  # 잔액 반환
            print("KRW Balance not found.")
        else:
            print("Failed to retrieve balance.")

    except Exception as e:
        print("Error Get Balance:", e)
        return None

bank_account = get_krw_balance()

KRW Balance: 29


In [407]:
# 자산 현황 조회
def get_candle(symbol, interval, limit, end):
    params = {
      "symbol" : str(symbol),
      "interval" : str(interval),
      "limit" : limit,
      "end" : end
    }
    query_string = urllib.parse.urlencode(params)

    signature = create_hmac_sha256_signature(SECRET_KEY, query_string)

    # 생성한 서명을 요청 변수에 추가
    params["signature"] = signature
    query_string = urllib.parse.urlencode(params)

    url = f"{base_url}/v2/candles?{query_string}"

    # 요청 헤더에 API 키 입력
    headers = {
        "X-KAPI-KEY": API_KEY
    }

    # HTTP GET 요청
    try:
        response = requests.get(url, headers=headers)
        return response.json()
    except Exception as e:
        print("Error Get Balance:", e)
        return None
end = int(time.time() * 1000)
print(end)
response = get_candle("btc_krw","60", 4, end)
print("Balance Response:", response)

1725009300325
Balance Response: {'success': True, 'data': [{'timestamp': 1724997600000, 'open': '79856000', 'high': '80520000', 'low': '79770000', 'close': '80520000', 'volume': '2.55546182'}, {'timestamp': 1725001200000, 'open': '80529000', 'high': '80763000', 'low': '80381000', 'close': '80400000', 'volume': '1.79707552'}, {'timestamp': 1725004800000, 'open': '80400000', 'high': '80720000', 'low': '80400000', 'close': '80600000', 'volume': '2.51917549'}, {'timestamp': 1725008400000, 'open': '80600000', 'high': '80750000', 'low': '80590000', 'close': '80750000', 'volume': '0.11652652'}]}


##    분봉 가져오기

In [412]:
now = datetime.datetime.now()
to = now.strftime('%Y-%m-%d %H:%M')

ticker = 'KRW-BTC'
interval = 'minute1'
count = 4
bitcoin_data = pyupbit.get_ohlcv(ticker=ticker, interval=interval, to=to, count=count)

open_change_list = ((bitcoin_data['open'] - bitcoin_data['open'].shift(1)) / bitcoin_data['open'].shift(1) * 100).round(2)
volume_list = bitcoin_data['volume'].round()
open_price = int(bitcoin_data['open'].iloc[-1])

print(bitcoin_data)

current_state = tuple(open_change_list[1:4]) + tuple(volume_list[1:3])
print(current_state)

                           open        high         low       close    volume  \
2024-08-30 18:12:00  80763000.0  80840000.0  80755000.0  80803000.0  1.019568   
2024-08-30 18:13:00  80840000.0  80865000.0  80803000.0  80865000.0  2.612569   
2024-08-30 18:14:00  80865000.0  80885000.0  80838000.0  80885000.0  2.027340   
2024-08-30 18:15:00  80845000.0  80885000.0  80845000.0  80880000.0  0.142112   

                            value  
2024-08-30 18:12:00  8.236832e+07  
2024-08-30 18:13:00  2.112153e+08  
2024-08-30 18:14:00  1.639560e+08  
2024-08-30 18:15:00  1.149033e+07  
(0.1, 0.03, -0.02, 3.0, 2.0)


##    예측

In [409]:
def sort_key(state_to_index):
    return (state_to_index[4], state_to_index[2], state_to_index[3], state_to_index[1], state_to_index[0])

def find_insert_index(sorted_states, new_state):
    new_state_key = sort_key(new_state)
    for index, state in enumerate(sorted_states):
        if sort_key(state) < new_state_key:
            return index
    return len(sorted_states)

def action_choose(bank_account):
  if bank_account > 10000:
    action_list = [0,1]
  else :
    action_list = [1,2]

  return action_list

Q_table = np.load('D:/StockQ-Learning/q_table_test.npy')

with open('states_to_index_test.pkl', 'rb') as f:
    states_to_index = pickle.load(f)

try:
    current_state_index = states_to_index[current_state]
except KeyError:
    current_state_index  = find_insert_index(states_to_index, current_state)
    if current_state_index >= len(states_to_index):
        current_state_index -= 1 

q_value = Q_table[current_state_index]
q_value_sorted = np.argsort(q_value)[::-1]
action_list = action_choose(bank_account)
print(action_list)
if q_value_sorted[0] in action_list:
    action = q_value_sorted[0]
elif q_value_sorted[0] not in action_list:
    if q_value_sorted[1] in action_list:
        action = q_value_sorted[1]
    elif q_value_sorted[1] not in action_list:
        action = q_value_sorted[2]

print(f'action : {action}')
print(f'bank_account : {bank_account}')
print(f'open_price : {open_price}')

[1, 2]
action : 1
bank_account : 29
open_price : 80865000


##    매수 함수

In [410]:
def place_order(symbol, side, price, qty, order_type, time_in_force):
    timestamp = int(time.time() * 1000)

    params = {
        "symbol": symbol,
        "side": side,
        "price": price,
        "qty": qty,
        "orderType": order_type,
        "timeInForce": time_in_force,
        "timestamp": str(timestamp)
    }
    query_string = urllib.parse.urlencode(params)
    
    signature = create_hmac_sha256_signature(SECRET_KEY, query_string)

    params["signature"] = signature
    
    url = f"{base_url}/v2/orders"
    headers = {
        "X-KAPI-KEY": API_KEY,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    try:
        response = requests.post(url, headers=headers, data=params)
        return response.json()
    except Exception as e:
        print("Error placing order:", e)
        return None

qty = int(bank_account) / open_price
qty = round(qty,8)
open_price = round((open_price*0.9995 // 1000)*1000)
print(open_price)

# response = place_order("btc_krw", "buy", str(open_price), str(qty), "limit", "gtc")
# print("Order response:", response)

80824000


##    거래하기

In [411]:
qty = int(bank_account) / open_price
qty = round(qty,8)
print(qty)

3.6e-07
